In [29]:
import os
import numpy as np
import pandas as pd
%matplotlib inline

In [30]:
def gpsReader(userDirectory, count):
    
    nameArray = []
    dfArray = []
    totalLoops = 0 
    path = os.walk(userDirectory)
    dir_in = userDirectory
    indexString = 0 #Tells the skiprow command how many rows to skip.
    
    for root, directories, files in path:
        for directory in directories:
            print(directory)
        for file in files:
            nameArray = files           #Fills the indices of the array with file names.
            totalLoops = totalLoops + 1 #Counts how many iterations the for loop completed.
            dfArray = files             #Creates the amount of indices needed for this array.
    
    if count == totalLoops: #This if statment stops the code from going past nameArray's index range.
        count = -1
        return 0, count
   
    elif count != totalLoops: #This if statment formats the gps data.
        file_in = nameArray[count]
        
        if nameArray[count].lower().endswith('.neu'):
            column_names=['Year', 'DN', 'DE', 'DU', 'SDN', 'SDE', 'SDU']
            
            if 'ULR6B' in nameArray[count]:       
                if 'dEYAC' in nameArray[count]:
                    indexString = 20          
                else:
                    indexString = 18
                
            elif 'JPL14' in nameArray[count]:
                    indexString = 30     
           
            elif 'ULR5' in nameArray[count]:
                indexString = 25
            
            elif 'NGL14' in nameArray[count]:
                if 'dKEN5' in nameArray[count]:
                    indexString = 34
                    
                elif 'dJNU1' in nameArray[count]:
                    indexString = 29
                    
                elif 'dNUUK' in nameArray[count]:
                    indexString = 23
                    
                elif 'dAKS' in nameArray[count]:
                    indexString = 21
                    
                elif 'dTBON' in nameArray[count]:
                    indexString = 27
                    
                elif 'dANC2' in nameArray[count]:
                    indexString = 26
                
            Df = pd.read_csv(os.path.join(dir_in, file_in), skiprows=indexString, header=None, delimiter='\s+', 
                             names=column_names)
            year = Df['Year'].astype(int)
            day = ((Df['Year'] - year) * 365).astype(int) + 1  # TODO: This might be off by one day...
            Dt = pd.to_datetime(year.astype(str) + day.astype(str), format='%Y%j')
            Df = Df.rename(columns={'Year': 'YearDec'})
            Df.index = pd.DatetimeIndex(Dt)
        
        
        elif nameArray[count].lower().endswith('.csv'):
            columns=['Date', 'North', 'East', 'Vertical', 'NorthSTD', 'EastSTD', 'VerticalSTD', 'Quality', 'NaN']
            Df = pd.read_csv(os.path.join(dir_in, file_in), skiprows=12, header=None, engine='c')
            if len(Df.columns) == len(columns):
                Df.columns = ['Date', 'North', 'East', 'Vertical', 'NorthSTD', 'EastSTD', 'VerticalSTD', 'Quality', 'NaN']
                Dt = pd.to_datetime(Df['Date'])
           
            # create datetime index  
            Df.index = pd.DatetimeIndex(Dt)
            Df = Df.drop('Date', axis=1)
            
        return Df, count

In [33]:
gpsArray = []  #The Array that will contain gps data
tempArray = [0] #Temporary array that will hold the gps data from the function
x = 0

while x != -1:
    tempArray[0], x = gpsReader('data/GPS', x)
    
    if x != -1:
        x = x + 1
        gpsArray.append(tempArray[0])
    

In [34]:
gpsArray[41]

,North,East,Vertical,NorthSTD,EastSTD,VerticalSTD,Quality,NaN
Date,,,,,,,,
2009-08-19,0.00,0.00,0.00,1.95,1.54,10.28,repro,NaN
2009-08-20,1.20,0.75,-9.75,2.11,1.60,11.36,repro,NaN
2009-08-21,2.11,1.60,0.81,2.03,1.60,10.65,repro,NaN
2009-08-22,1.65,1.75,5.62,2.05,1.65,10.92,repro,NaN
2009-08-23,1.15,2.04,4.69,1.99,1.60,10.52,repro,NaN
...,...,...,...,...,...,...,...,...
2020-06-20,-1.06,-34.21,13.48,3.77,2.64,19.14,rapid,NaN
2020-06-21,0.66,-32.90,9.73,3.82,2.67,19.48,rapid,NaN
2020-06-22,-1.16,-34.29,15.21,3.85,2.69,19.80,rapid,NaN


In [27]:
def test_find_skiprows_startofline(infile, search_string, diagnostics=False):
    '''

    Parameters
    ----------
    infile
    search_string
    diagnostics
    
    Returns
    -------
    i
    '''
    with open(infile) as fp:
        for i, line in enumerate(fp):
            if line[:len(search_string)] == search_string:
                if diagnostics:
                    print(line)
                    print(len(search_string))
                return i
            
            
i_skip = test_find_skiprows_startofline('data/GPS/dKEN5_XXXXXXXXX_NGL14.neu', '#  Year')
print(i_skip)

32
